In [47]:
import pandas as pd
import math
import numpy as np

data = pd.read_csv("C:/Users/sanja/4NM21MC085/Assignment 8/weather.csv")
features = [feat for feat in data]
features.remove("Played football(yes/no)")

In [48]:
class Node:
    def __init__(self):
        self.children = []
        self.value = ""
        self.isLeaf = False
        self.pred = ""

In [49]:
def entropy(examples):
    pos = 0.0
    neg = 0.0
    for _, row in examples.iterrows():
        if row["Played football(yes/no)"] == "Yes":
            pos += 1
        else:
            neg += 1
    if pos == 0.0 or neg == 0.0:
        return 0.0
    else:
        p = pos / (pos + neg)
        n = neg / (pos + neg)
        return -(p * math.log(p, 2) + n * math.log(n, 2))

In [50]:
def info_gain(examples, attr):
    uniq = np.unique(examples[attr])
    #print ("\n",uniq)
    gain = entropy(examples)
    #print ("\n",gain)
    for u in uniq:
        subdata = examples[examples[attr] == u]
        print ("\n",subdata)
        sub_e = entropy(subdata)
        gain -= (float(len(subdata)) / float(len(examples))) * sub_e
        print ("\n",gain)
    return gain

In [51]:
def ID3(examples, attrs):
    root = Node()

    max_gain = 0
    max_feat = ""
    for feature in attrs:
        #print ("\n",examples)
        gain = info_gain(examples, feature)
        if gain > max_gain:
            max_gain = gain
            max_feat = feature
    root.value = max_feat
    #print ("\nMax feature attr",max_feat)
    uniq = np.unique(examples[max_feat])
    #print ("\n",uniq)
    for u in uniq:
        #print ("\n",u)
        subdata = examples[examples[max_feat] == u]
        #print ("\n",subdata)
        if entropy(subdata) == 0.0:
            newNode = Node()
            newNode.isLeaf = True
            newNode.value = u
            newNode.pred = np.unique(subdata["Played football(yes/no)"])
            root.children.append(newNode)
        else:
            dummyNode = Node()
            dummyNode.value = u
            new_attrs = attrs.copy()
            new_attrs.remove(max_feat)
            child = ID3(subdata, new_attrs)
            dummyNode.children.append(child)
            root.children.append(dummyNode)

    return root

In [52]:
def printTree(root: Node, depth=0):
    for i in range(depth):
        print("\t", end="")
    print(root.value, end="")
    if root.isLeaf:
        print(" -> ", root.pred)
    print()
    for child in root.children:
        printTree(child, depth + 1)

In [53]:
def classify(root: Node, new):
    for child in root.children:
        if child.value == new[root.value]:
            if child.isLeaf:
                print ("Predicted Label for new example", new," is:", child.pred)
                exit
            else:
                classify (child.children[0], new)

In [54]:
root = ID3(data, features)
print("Decision Tree is:")
printTree(root)
print ("------------------")

new = {"Outlook":"Rain", "Temperature":"Hot", "Humidity":"Normal", "Wind":"Weak"}
classify (root, new)


      Outlook Temperature Humidity    Wind Played football(yes/no)
2   Overcast         Hot     High    Weak                     Yes
6   Overcast        Cool   Normal  Strong                     Yes
11  Overcast        Mild     High  Strong                     Yes
12  Overcast         Hot   Normal    Weak                     Yes

 0.9402859586706309

    Outlook Temperature Humidity    Wind Played football(yes/no)
3     Rain        Mild     High    Weak                     Yes
4     Rain        Cool   Normal    Weak                     Yes
5     Rain        Cool   Normal  Strong                      No
9     Rain        Mild   Normal    Weak                     Yes
13    Rain        Mild     High  Strong                      No

 0.593517889222535

    Outlook Temperature Humidity    Wind Played football(yes/no)
0    Sunny         Hot     High    Weak                      No
1    Sunny         Hot     High  Strong                      No
7    Sunny        Mild     High    Weak        